# Chapter 4

Bayesian interpretations of linear regression, via some Gaussians

In [ ]:
import arviz
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import seaborn as sns
import pandas as pd
import pymc as pm 

import pybayes

sns.set_style("white") 

In [ ]:
%load_ext watermark
%watermark -v -m -p arviz,matplotlib,numpy,scipy,seaborn,pandas,pymcs


Firstly, we simulate some random walks to show empirically (if further evidence were needed) that lots of things end up being Gaussian.

Eg random walks via binomial (e.g. flip a coin, step forward if heads, backward if tails).

In [ ]:
n_walks = 1000
n_steps = 1000

walks = np.random.binomial(n=1, p=0.5, size=(n_walks, n_steps))
walks[walks == 0] = -1

paths = np.cumsum(walks, axis=1)

fig, ax = plt.subplots()

for i in range(n_walks):
    plt.plot(paths[i, :], alpha=0.05)

plt.title('Walks')
plt.xlabel('Step number')
plt.ylabel('Position')
plt.show()

In [ ]:
final_position = paths[:, -1]
pybayes.utils.hist(final_position)
plt.show()

## Grid-approximating our two-parameter model

We're going to do a regression on some height data. Our model will be:

\begin{equation}
\begin{aligned}
h_i &\sim \mathcal{N}(\mu, \sigma) \\
\mu &\sim \mathcal{N}(178, 20) \\
\sigma &\sim \text{Uniform}(0, 50)\end{aligned}
\end{equation}

Here our likelihood is line one, and line two and three are sensibly chosen priors. We can check the sensibleness by plotting the priors, and then looking at what they imply, with a prior predictive simulation

In [ ]:
mu_mean = 178
mu_sigma = 20
p_grid_mu = np.linspace(100,250, 1000)
mu_prior = scipy.stats.norm.pdf(p_grid_mu, loc= mu_mean, scale=mu_sigma)

In [ ]:
pybayes.utils.plot_nicely(x_vals=p_grid_mu, y_vals=mu_prior)

In [ ]:
sigma_low = 0
sigma_high = 50
p_grid_sigma = np.linspace(-5,55, 100)
sigma_prior = scipy.stats.uniform.pdf(p_grid_sigma, loc=sigma_low, scale=sigma_high)
# it is really weird that uniform uses loc and scale to mean these things.
pybayes.utils.plot_nicely(x_vals=p_grid_sigma, y_vals=sigma_prior)

In [ ]:
sample_mu = np.random.normal(loc=mu_mean, scale=mu_sigma, size=10_000)
sample_sigma = np.random.uniform(low=sigma_low, high=sigma_low, size=10_000)
prior_h = np.random.normal(loc=sample_mu, scale=sample_sigma)

In [ ]:
pybayes.utils.hist(prior_h)

The above is not the empirical distribution of H, its not even Gaussian. It's the distribution of relative plausibilities of different heights before we've seen the data. Next step is to grab the data and grid-approximate the posterior.

In [ ]:
howell = "https://raw.githubusercontent.com/rmcelreath/rethinking/master/data/Howell1.csv"

df = pd.read_csv(howell, sep=';')

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
# only use adults here
d2 = df[df.age >= 18]

In [ ]:
# grid-approximate using an algo to be explained later.
mu_list = np.linspace(150, 160, 100)
sigma_list = np.linspace(7,9, 100)  # why these values i do not know - i assume this is from the observed data.
# all combos of sigma and mu
post = pd.DataFrame({
    'mu': np.tile(mu_list, len(sigma_list)),
    'sigma': np.repeat(sigma_list, len(mu_list))
}) 

# Calculate the log likelihoods
def log_likelihood(row):
    mu = row['mu']
    sigma = row['sigma']
    ll = np.sum(scipy.stats.norm.logpdf(d2['height'], mu, sigma))
    return ll

post['LL'] = post.apply(log_likelihood, axis=1)

# Calculate the product of likelihood and priors
post['prod'] = (post['LL'] + 
                scipy.stats.norm.logpdf(post['mu'], 178, 20) + 
                np.where((post['sigma'] >= 0) & (post['sigma'] <= 50), np.log(1/50), -np.inf))

# Convert to probability
max_prod = np.max(post['prod'])
post['prob'] = np.exp(post['prod'] - max_prod)

In [ ]:
sns.kdeplot(data=post, x='mu', y='sigma', weights='prob',  fill=True)

In [ ]:
two_d = post.pivot(index='mu', columns='sigma', values='prob')

In [ ]:
sns.heatmap(two_d)

In [ ]:
# sample from the posterior, by sampling from the rows numbers proportionally to the probability and pulling the params.

rows = np.random.choice(post.index, 10_000, replace=True, p=post.prob/post.prob.sum())


sample = pd.DataFrame.from_dict({'mu': post.iloc[rows].mu,
                                 'sigma': post.iloc[rows].sigma}).reset_index(drop=True)

In [ ]:
sample

In [ ]:
sns.scatterplot(data=sample, x='mu', y='sigma', alpha=0.05)

In [ ]:
sns.histplot(sample.mu, binwidth=0.1)

In [ ]:
sns.histplot(sample.sigma, binwidth=0.1)

In [ ]:
# recall our priors for mu and sigma 
fig, ax = plt.subplots()
ax.plot(p_grid_mu, mu_prior, label='prior')
ax.set_ylabel('p')
ax2 = ax.twinx()
sns.histplot(sample.mu, binwidth=0.1, ax=ax2, label='posterior')

plt.show()

In [ ]:
# our posterior for mu has collapsed as a result of our observations.
print(f'HDPI for mu:', arviz.hdi(sample.mu.values))
print(f'HDPI for sigma:', arviz.hdi(sample.sigma.values))

In [ ]:
d3 = d2.sample(20)

In [ ]:
# If we repeat all the above but only using 20 of the heights from the dataset, we get:
# grid-approximate using an algo to be explained later.
mu_list = np.linspace(150, 170, 100)
sigma_list = np.linspace(4,20, 100)  
# all combos of sigma and mu
post = pd.DataFrame({
    'mu': np.tile(mu_list, len(sigma_list)),
    'sigma': np.repeat(sigma_list, len(mu_list))
}) 

# Calculate the log likelihoods
def log_likelihood(row):
    mu = row['mu']
    sigma = row['sigma']
    ll = np.sum(scipy.stats.norm.logpdf(d3['height'], mu, sigma))
    return ll

post['LL'] = post.apply(log_likelihood, axis=1)

# Calculate the product of likelihood and priors
post['prod'] = (post['LL'] + 
                scipy.stats.norm.logpdf(post['mu'], 178, 20) + 
                np.where((post['sigma'] >= 0) & (post['sigma'] <= 50), np.log(1/50), -np.inf))

# Convert to probability
max_prod = np.max(post['prod'])
post['prob'] = np.exp(post['prod'] - max_prod)

# sample
rows = np.random.choice(post.index, 10_000, replace=True, p=post.prob/post.prob.sum())
sample = pd.DataFrame.from_dict({'mu': post.iloc[rows].mu,
                                 'sigma': post.iloc[rows].sigma}).reset_index(drop=True)

In [ ]:
sns.histplot(sample.mu, binwidth=0.1)

In [ ]:
sns.histplot(sample.sigma, binwidth=0.1)

In [ ]:
sns.scatterplot(data=sample, x='mu', y='sigma', alpha=0.05)

In [ ]:
# the stdev is notably less Gaussian.

## Moving to quadratic approximation

In r this is all done in quap. We fit a quadratic to the maximum of the a posteriori distro and use that.

The model again:\begin{equation}
\begin{aligned}
h_i &\sim \mathcal{N}(\mu, \sigma) \\
\mu &\sim \mathcal{N}(178, 20) \\
\sigma &\sim \text{Uniform}(0, 50)\end{aligned}
\end{equation}

In [ ]:
d2

In [ ]:
# TODO: make and use py-quap, this is doing MCMC sampling

with pm.Model() as height_model:
    # Uniform prior for sigma
    mu = pm.Normal('mu', mu=178, sigma=20)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    
    # Normal likelihood
    height = pm.Normal('height', mu=mu, sigma=sigma, observed=d2.height)
    

In [ ]:
with height_model:
    trace = pm.sample(1000, tune=1000)

In [ ]:
arviz.plot_trace(trace)
plt.tight_layout()
plt.show()

In [ ]:
print(arviz.summary(trace, kind='stats'))

In [ ]:
# what happens if we use a much tighter and more informative prior on mu?
with pm.Model() as height_model_2:
    # Uniform prior for sigma
    mu = pm.Normal('mu', mu=178, sigma=0.1)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    
    # Normal likelihood
    height = pm.Normal('height', mu=mu, sigma=sigma, observed=d2.height)
    
    
    trace_2 = pm.sample(1000, tune=1000)
    
arviz.plot_trace(trace_2)
plt.tight_layout()
plt.show()
print(arviz.summary(trace_2, kind='stats'))

In [ ]:
# our model is insistent that the mean is 178, and this disagrees with the data a lot, so the posterior for sigma changes

In [ ]:
# sampling from the quadratic approximation
# note that here we're already got samples, but pretend we used quap. Then the quadratic approximation 
# is a multi-dimensional Gaussian, specified by the means and covariance of our distro.
trace_df = arviz.extract(trace, combined=True).to_dataframe()

In [ ]:
trace_df[['mu', 'sigma']].cov()

In [ ]:
# decompose into the variances for the params, and the correlation
np.diag(trace_df[['mu', 'sigma']].cov())

In [ ]:
trace_df[['mu', 'sigma']].corr()

In [ ]:
# this matrix shows that learning about mu tells us little about sigma, and vice versa - may not always be the case.

In [ ]:
# we can extract vectors of values from the Gaussian, given this info.
mean_values = trace_df.mean()[['mu', 'sigma']]
quad_samples = scipy.stats.multivariate_normal.rvs(mean=mean_values, cov=trace_df[['mu', 'sigma']].cov(), size=1000)

In [ ]:
pybayes.utils.hist(quad_samples[:,0])

In [ ]:
pybayes.utils.hist(quad_samples[:,1])

In [ ]:
arviz.hdi(quad_samples[:,0])

In [ ]:
arviz.hdi(quad_samples[:,1])

## Predicting things

We've fit a Gaussian to some heights. What we want to do is model how some predictor variables affect an outcome of interest.
Here we'll use weight to predict height.

\begin{equation}
\begin{aligned}
h_i &\sim \mathcal{N}(\mu_i, \sigma) \\
\mu_i &= \alpha + \beta(x_i - \bar{x}) \\
\alpha &\sim \text{Normal}(178, 20) \\
\beta  &\sim \text{Normal}(0,10) \\
\sigma &\sim \text{Uniform}(0, 50)
\end{aligned}
\end{equation}

Now the mean depends on each row $i$. And we no longer estimate $\mu$ as a parameter, instead we construct it, assuming the linear model given. Note the lack of $\sim$, the $\mu_i$ is deterministic given the inputs.

In [ ]:
sns.scatterplot(data=d2, x='weight', y='height')

In [ ]:
# what do our priors mean? We can do a prior predictive simulation

N = 100
alpha = np.random.normal(loc=178, scale=20, size=N)
beta = np.random.normal(loc=0, scale=10, size=N)

fig, ax= plt.subplots()
x = np.linspace(30, 60, N)
x_bar = d2.weight.mean()
for a, b in zip(alpha, beta):
    
    ax.plot(x, [a + b*(i-x_bar) for i in x], alpha=0.1)
    
plt.show()

note this is very silly. Noone on Earth is <0 or > 300 cm tall. So use a new prior on beta:

\begin{equation}
\beta  \sim \text{Log-Normal}(0,1) \\
\end{equation}

In [ ]:
beta = np.random.lognormal(mean=0, sigma=1, size=10_000)
sns.histplot(beta)

In [ ]:
# Repeat our prior predictive simulation

N = 100
alpha = np.random.normal(loc=178, scale=20, size=N)
beta = np.random.lognormal(mean=0, sigma=1, size=N)

fig, ax= plt.subplots()
x = np.linspace(30, 60, N)
x_bar = d2.weight.mean()
for a, b in zip(alpha, beta):
    ax.plot(x, [a + b*(i-x_bar) for i in x], alpha=0.1)

plt.show()

In [ ]:
# now generate the posterior, as before

x_bar = d2.weight.mean()

with pm.Model() as height_model_2:
    # Uniform prior for sigma
    alpha = pm.Normal('alpha', mu=178, sigma=20)
    beta = pm.LogNormal('beta', mu=0, sigma=1)
    sigma = pm.Uniform('sigma', lower=0, upper=50)

    mu = alpha + beta*(d2.weight-x_bar)
    
    height=pm.Normal('height', mu=mu, sigma=sigma, observed=d2.height) 
    
    trace_regr = pm.sample(1000, tune=1000)

In [ ]:
arviz.plot_trace(trace_regr)
plt.tight_layout()
plt.show()
print(arviz.summary(trace_regr, kind='stats'))

In [ ]:
# visualising our posterior. To start with, look at the raw data and the posterior mean.
trace_regr_df = trace_regr.posterior.to_dataframe()
fix, ax = plt.subplots()
ax.scatter(data=d2, x='weight', y='height', alpha=0.25)

x = np.linspace(d2.weight.min(), d2.weight.max(), 100)
alpha_mean = trace_regr_df.alpha.mean()
beta_mean = trace_regr_df.beta.mean()
y = alpha_mean + beta_mean*(x - x_bar)

ax.plot(x, y)

plt.show()


In [ ]:
# now add some sample lines to show the uncertainty in the parameter values
trace_regr_df = trace_regr.posterior.to_dataframe()
fix, ax = plt.subplots()
ax.scatter(data=d2, x='weight', y='height', alpha=0.25)

x = np.linspace(d2.weight.min(), d2.weight.max(), 100)
alpha_mean = trace_regr_df.alpha.mean()
beta_mean = trace_regr_df.beta.mean()
y = alpha_mean + beta_mean*(x - x_bar)

ax.plot(x, y, c='green')

lines = trace_regr_df.sample(10)

for _, (a, b, s) in lines.iterrows():
    y = a + b*(x - x_bar)
    ax.plot(x, y, c='green', alpha=.2)
# for i in range(10):
#     alpha

plt.show()



In [ ]:
# we can see, as the number of points we're inferring from increases, the uncertainty is reduced.

def get_posterior_from_sample(input_df) -> pd.DataFrame:
    x_bar = input_df.weight.mean()
    with pm.Model() as height_model_3:
        alpha = pm.Normal('alpha', mu=178, sigma=20)
        beta = pm.LogNormal('beta', mu=0, sigma=1)
        sigma = pm.Uniform('sigma', lower=0, upper=50)
        mu = alpha + beta*(input_df.weight-x_bar)
        height=pm.Normal('height', mu=mu, sigma=sigma, observed=input_df.height) 
        trace_regr = pm.sample(1000, tune=1000)
    return trace_regr.posterior.to_dataframe()

for num_points in [10, 10, 100, len(d2)]:
    sub_df = d2[:num_points]
    # now generate the posterior, as before
    posterior = get_posterior_from_sample(sub_df)
    fix, ax = plt.subplots()
    ax.scatter(data=sub_df, x='weight', y='height', alpha=0.25)

    x = np.linspace(30, 65, 100)
    alpha_mean = posterior.alpha.mean()
    beta_mean = posterior.beta.mean()
    y = alpha_mean + beta_mean*(x - x_bar)
    ax.plot(x, y, c='green')

    lines = posterior.sample(10)
    for _, (a, b, s) in lines.iterrows():
        y = a + b*(x - x_bar)
        ax.plot(x, y, c='green', alpha=.2)

    plt.show()


In [ ]:
# we can do better by plotting the interval.

# to start with, what's the distribution of the posterior mu at a fixed point (e.g. weight=50)?

mu_at_50 = trace_regr_df.alpha + trace_regr_df.beta * (50 - x_bar)

sns.kdeplot(mu_at_50)
plt.xlabel('mu | weight=50')

In [ ]:
# mu has a distribution (Gaussian, since its inputs are all Gaussians). So we can work out the HPDI.
arviz.hdi(mu_at_50.values, hdi_prob=0.89)

In [ ]:
trace_regr_df.alpha.values

In [ ]:
# we can draw this interval for each value of the weight
x = np.linspace(d2.weight.min(), d2.weight.max(), 100)
y= trace_regr_df.alpha.values[:, np.newaxis] + trace_regr_df.beta.values[:, np.newaxis] * (x - x_bar).T


In [ ]:
fig, ax = plt.subplots()
arviz.plot_hdi(x, y)
ax.scatter(data=d2, x='weight', y='height', alpha=0.25)
alpha_mean = trace_regr_df.alpha.mean()
beta_mean = trace_regr_df.beta.mean()
y = alpha_mean + beta_mean*(x - x_bar)
ax.plot(x, y)

plt.show()


In [ ]:
# what we want, though, are prediction intervals for h, not for mu.

# redo the sampling from the posterior, just for convenience
x_bar = d2.weight.mean()
with pm.Model() as model_simulate_h:
    alpha = pm.Normal('alpha', mu=178, sigma=20)
    beta = pm.LogNormal('beta', mu=0, sigma=1)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    mu = alpha + beta*(d2.weight-x_bar)
    height=pm.Normal('height', mu=mu, sigma=sigma, observed=d2.height) 
    samples = pm.sample(1000, tune=1000)

samples_df = samples.posterior.to_dataframe()


In [ ]:
len(samples_df)

In [ ]:
# we could sample heights and get hpdis by drawing from the normal distribution with the rows from samples_df. But
# its easier to use pymc's builtins. Let's do the former this time to dispel the magic.
sim_weights = np.linspace(d2.weight.min(), d2.weight.max(), 100)

hpdis = np.zeros((len(samples_df), len(sim_weights))) # [[] * len(sim_weights)]

for i, sim_weight in enumerate(sim_weights):
    mu = samples_df.alpha + samples_df.beta*(sim_weight-d2.weight.mean())  # 4000 mu values
    heights = np.random.normal(loc=mu, scale=samples_df.sigma) # 4000 heights
    hpdis[:, i] = heights

In [ ]:
# draw everything
fig, ax = plt.subplots()

# raw data
ax.scatter(data=d2, x='weight', y='height', alpha=0.25, color='#7570b3')

# MAP line
x = np.linspace(d2.weight.min(), d2.weight.max(), 100)
alpha_mean = trace_regr_df.alpha.mean()
beta_mean = trace_regr_df.beta.mean()
y = alpha_mean + beta_mean*(x - x_bar)
ax.plot(x, y, color='#1b9e77')

# HPDI for the line
mus= trace_regr_df.alpha.values[:, np.newaxis] + trace_regr_df.beta.values[:, np.newaxis] * (x - x_bar).T
arviz.plot_hdi(x, mus, hdi_prob=0.89, fill_kwargs={'alpha': 0.25, 'color': '#1b9e77'})

# HPDI for simulated heights
arviz.plot_hdi(x, hpdis, hdi_prob=0.89, fill_kwargs={'alpha': 0.1, 'color': '#1b9e77'})


plt.show()


## Linear regression with curves

We can use linear regression with more complicated models. E.g:


\begin{equation}
\mu_i = \alpha + \beta_1 x_i - \beta_2 x_i^2 
\end{equation}

In [ ]:
# look at our dataset, now including children. Note that it is absolutely not linear.
df.plot.scatter(x='weight', y='height')

### Parabolae


\begin{equation}
\begin{aligned}
h_i &\sim \mathcal{N}(\mu_i, \sigma) \\
\mu_i &= \alpha + \beta_1 x_i - \beta_2 x_i^2  \\
\alpha &\sim \text{Normal}(178, 20) \\
\beta_1  &\sim \text{Log-Normal}(0,1) \\
\beta_2 &\sim \text{Normal}(0,1) \\
\sigma &\sim \text{Uniform}(0, 50)
\end{aligned}
\end{equation}

The prior for $\beta_2$ is somewhat arbitrarily assigned here. Prior predictive simulation will help guide the way (see practice problems at the end)

In [ ]:
# approximate the posterior, with standardised weights
df['weight_s'] = (df.weight - df.weight.mean()) / df.weight.std()
df['weight_s2'] = df.weight_s.pow(2)

with pm.Model() as poly_model:
    # priors
    alpha = pm.Normal('alpha', mu=178, sigma=20)
    beta1 = pm.LogNormal('beta1', mu=0, sigma=1)
    beta2 = pm.Normal('beta2', mu=0, sigma=1)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    # model
    mu = pm.Deterministic('mu', alpha + beta1 * df.weight_s + beta2 * df.weight_s2)
    # likelihood
    height=pm.Normal('height', mu=mu, sigma=sigma, observed=df.height) 
    poly_model_samples = pm.sample(1000, tune=1000)

In [ ]:
arviz.plot_trace(poly_model_samples, var_names=['~mu'])
plt.tight_layout()
plt.show()

In [ ]:
arviz.summary(poly_model_samples, var_names=['~mu'], hdi_prob=.89).round(2)

In [ ]:
poly_model_posterior = poly_model_samples.posterior.to_dataframe()

In [ ]:
poly_model_posterior.describe().T

In [ ]:
mu_pred = poly_model_samples.posterior['mu']

height_pred = pm.sample_posterior_predictive(poly_model_samples, model=poly_model)

In [ ]:
# plot the mean relationship and the 89% intervals
fig, ax = plt.subplots()

# raw data
ax.scatter(data=df, x='weight_s', y='height', alpha=0.25, color='#7570b3')
s
# MAP line
# num_x_vals = 100
# x = np.linspace(df.weight_s.min(), df.weight_s.max(), num_x_vals)
# alpha_mean = poly_model_posterior.alpha.mean()
# beta1_mean = poly_model_posterior.beta1.mean()
# beta2_mean = poly_model_posterior.beta2.mean()


# ppd interval for the mean
num_x_vals = 100
x = np.linspace(df.weight_s.min(), df.weight_s.max(), num_x_vals)

arviz.plot_hdi(df.weight_s, mu_pred, hdi_prob=0.89, fill_kwargs={'alpha': 0.25, 'color': '#1b9e77'})
# ppd interval for the heights
arviz.plot_hdi(df.weight_s, height_pred.posterior_predictive['height'], hdi_prob=0.89, fill_kwargs={'alpha': 0.1, 'color': '#1b9e77'})


plt.show()


In [ ]:
# polynomial model definitely fits the sample data better than a linear model - does that make it a  better model? 
# are we learning anything causal from this?

### Splines

Specifically b-splines, using basis functions. We will look at cherry blossom data since it's wigglier.

In [ ]:
cherry_blossom_url = "https://raw.githubusercontent.com/rmcelreath/rethinking/master/data/cherry_blossoms.csv"
cherry_blossoms = pd.read_csv(cherry_blossom_url, sep=';')

In [ ]:
cherry_blossoms.describe().T

In [ ]:
# here doy is the day of the year of the first day of blossom

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
sns.scatterplot(data=cherry_blossoms, x='year', y='doy')
plt.show()

With B-splines, we generate new predictor variables and use those in our linear model $\mu_i$, using basis functions $B_i$

Our model then becomes:
\begin{equation}
\mu_i = \alpha + w_1 B_{i1} + w_2 B_{i2} + ...
\end{equation}

i.e. we have a w parameter for each basis function. Our basis variables can be constructed in a bunch of ways - here we use linear basis functions that turn off and on such that at any point only two basis functions are non-zero. We call the pivot points at which the basis functions peak the 'knots'

In [ ]:
# choose the knots/pivot points - here we use quantiles of the year
cherry_blossoms_non_null = cherry_blossoms[cherry_blossoms.doy.notnull()]
num_knots = 15
knot_locations = cherry_blossoms_non_null.year.quantile(np.linspace(0, 1, num_knots)).values.astype(int)
internal_knots = knot_locations[1:-1]

In [ ]:
# this actually does that fitting - to do the matrix-style approach i'd need the 'patsy' library which I've never used before,
# so i shan't.
spline = scipy.interpolate.LSQUnivariateSpline(cherry_blossoms_non_null.year, cherry_blossoms_non_null.doy, t=internal_knots, k=3)
fig, ax = plt.subplots(figsize=(15,5))
sns.scatterplot(data=cherry_blossoms, x='year', y='doy', alpha=0.25)
plt.plot(cherry_blossoms_non_null.year, spline(cherry_blossoms_non_null.year))
plt.show()


In [ ]:
spline = scipy.interpolate.LSQUnivariateSpline(cherry_blossoms_non_null.year, cherry_blossoms_non_null.doy, t=internal_knots, k=1)
fig, ax = plt.subplots(figsize=(15,5))
sns.scatterplot(data=cherry_blossoms, x='year', y='doy', alpha=0.25)
plt.plot(cherry_blossoms_non_null.year, spline(cherry_blossoms_non_null.year))
plt.show()


In [ ]:
# to fit the model and get the posterior we'd need the actual basis functions, which sadly we do not have.

## Solutions to exercises

Do the exercises yourself.

- 4E1: line 1
- 4E2: 2
- 4E3: p(mu, sigma | y) = p(y | sigma, mu) p (mu | 0, 10) p (sigma | 1) / big interval
- 4E4: 2
- 4E5: 3 (mu is now deterministic)

4M1 - simulate observed y values from the prior

\begin{equation}
\begin{aligned}
y_i &\sim \mathcal{N}(\mu, \sigma) \\
\mu &\sim \text{Normal}(0, 10) \\
\sigma &\sim \text{Exponential}(1)
\end{aligned}
\end{equation}


In [ ]:
num_samples = 10_000
mu = np.random.normal(loc=0, scale=10, size=num_samples)
sigma = np.random.exponential(scale=1, size=num_samples)
y = np.random.normal(mu, sigma)

fig, axes = plt.subplots(ncols=3, figsize=(15,5))

for sample, ax, name in zip([mu, sigma, y], axes, ['mu', 'sigma', 'y']):
    sns.histplot(sample, ax=ax)
    ax.set_title(name)

plt.show()

In [ ]:
# 4M2: as above, but with quap
# (i will be using pymc4 because i haven't implemented pyquap yet
with pm.Model() as model_4m2:
    # priors
    mu = pm.Normal('mu', mu=0, sigma=10)
    sigma = pm.Exponential('sigma', lam=1)
    # likelihood
    y=pm.Normal('y', mu=mu, sigma=sigma) 

4M3

\begin{equation}
\begin{aligned}
y_i &\sim \mathcal{N}(\mu_i, \sigma) \\
\mu_i &= \alpha + \beta x \\
\alpha &\sim \text{Normal}(0, 10) \\
\beta &\sim \text{Uniform}(0, 1) \\
\sigma &\sim \text{Exponential}(1)
\end{aligned}
\end{equation}


**4M4: measure height each year for 3 years. Predict height using year as a predictor. What's the model definition?**
\begin{equation}
\begin{aligned}
h_ij &\sim \mathcal{N}(\mu_{}, \sigma) \\
\mu_i &= \alpha + \beta (y_i - \bar{y})  \\
\alpha &\sim \text{Normal}(100, 20) \\
\beta &\sim \text{Log-Normal}(1, 1) \\
\sigma &\sim \text{Exponential}(1)
\end{aligned}
\end{equation}

We could do prior predictives on this to see if it looked reasonable.

**4M5: does knowing that every student gets taller each year change the priors?**

No - this is factored into the prior choice for $\beta$ already.

**4M6: what about if you know that the variance among heights is never more than 64cm?**

If we know $\sigma^2 < 64$ then maybe a uniform distribution is better than an exponential, $\sigma \sim \text{Uniform}(0, 8)$. What proportion of an exponential with scale 1 is >8, anyway?

In [ ]:
samples = np.random.exponential(scale=1, size=100_000)
sns.histplot(samples)

In [ ]:
num_greater_than_8 = (samples > 8).sum()
print(f'proportion of exponential distribution > 8: {num_greater_than_8/len(samples)*100:.2f}%')

In [ ]:
# 4M7: refit m4.3, without including the mean weight $\bar{x}$. Compare the posterior, and the covariance. Then look at the posterior predictions.
# (expectation - they're the same, the covariance is higher because of the lack of scaling).

with pm.Model() as model_4m7:
    # priors
    alpha = pm.Normal('alpha', mu=178, sigma=20)
    beta = pm.LogNormal('beta', mu=0, sigma=1)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    # model
    mu = pm.Deterministic('mu', alpha + beta * d2.weight)
    # likelihood
    height=pm.Normal('height', mu=mu, sigma=sigma, observed=d2.height) 
    model_4m7_samples = pm.sample(1000, tune=1000)
    mean_q = pm.find_MAP()
    hess = pm.find_hessian(mean_q, vars=[alpha, beta, sigma])    

    

In [ ]:
mu_pred = model_4m7_samples.posterior['mu']
height_pred = pm.sample_posterior_predictive(model_4m7_samples, model=model_4m7)

In [ ]:
x = np.linspace(d2.weight.min(), d2.weight.max(), 100)
mu = mean_q['alpha'] + mean_q['beta'] * x

plt.plot(x, mu)
plt.scatter(d2.weight, d2.height, alpha=0.25)

arviz.plot_hdi(d2.weight, mu_pred, hdi_prob=0.89, fill_kwargs={'alpha': 0.25, 'color': '#1b9e77'})
# ppd interval for the heights
arviz.plot_hdi(d2.weight, height_pred.posterior_predictive['height'], hdi_prob=0.89, fill_kwargs={'alpha': 0.1, 'color': '#1b9e77'})

plt.show()

In [ ]:
np.set_printoptions(suppress=True)
np.linalg.inv(hess)

In [ ]:
sns.histplot(height_pred.posterior_predictive['height'].to_dataframe())

(skipping the splines q, 4M*)

4H1: given new data points, weights: 46.95, 43.72, 64.78, 32.59, 54.63. Give the expected height and 89% interval.

In [ ]:
x_bar = d2.weight.mean()

with pm.Model() as model_4h1:
    # priors
    alpha = pm.Normal('alpha', mu=178, sigma=20)
    beta = pm.LogNormal('beta', mu=0, sigma=1)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    # model
    mu = pm.Deterministic('mu', alpha + beta * (d2.weight - x_bar))
    # likelihood
    height=pm.Normal('height', mu=mu, sigma=sigma, observed=d2.height) 
    model_4h1_samples = pm.sample(1000, tune=1000)
    map_h = pm.find_MAP()

                          

In [ ]:
model_4h1_samples_df = model_4h1_samples.posterior.to_dataframe()

In [ ]:
new_weights = [46.95, 43.72, 64.78, 32.59, 54.63]
fig, axes = plt.subplots(figsize=(5, 20), sharex=True, nrows=len(new_weights))
new_weights_intervals = []
new_weights_means = []
for new_weight, ax in zip(new_weights, axes):
    # sample the posterior. 
    mu = model_4h1_samples_df.alpha + model_4h1_samples_df.beta*(new_weight-d2.weight.mean())  # a bunch of possible mus
    heights = np.random.normal(loc=mu, scale=model_4h1_samples_df.sigma)
    sns.histplot(heights, ax=ax)
    ax.set_title(f'Posterior prediction of heights for weight: {new_weight}')
    # use mean as MAP approximation
    print(f'hdi for {new_weight=}: {arviz.hdi(heights, hdi_prob=0.89)}')
    print(f'mean of posterior for {new_weight=}: {heights.mean()}')
plt.show()

In [ ]:
# 4H2 - look at only ages <= 18 in the Howell1 data. 
# a) fit a linear regression, present the estimates. For every 10 units of increase in weight, what is the predicted increase in height?
# b) plot the raw data. Show the MAP line and the 89% interval for the mean, and predicted heights.
# c) what's wrong with your model?

In [ ]:
howell_children = df[df.age < 18]

In [ ]:
x_bar = howell_children.weight.mean()

with pm.Model() as model_4h2:
    # priors
    alpha = pm.Normal('alpha', mu=100, sigma=20)
    beta = pm.LogNormal('beta', mu=0, sigma=1)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    # model
    mu = pm.Deterministic('mu', alpha + beta * (howell_children.weight - x_bar))
    # likelihood
    height=pm.Normal('height', mu=mu, sigma=sigma, observed=howell_children.height) 
    model_4h2_samples = pm.sample(1000, tune=1000)
    map_vals = pm.find_MAP()

                          

In [ ]:
float(map_vals['alpha'])

In [ ]:
print('model alpha:', round(float(map_vals['alpha']), 2))
print('model beta:', round(float(map_vals['beta']), 2))
print('model sigma:', round(float(map_vals['sigma']), 2))

In [ ]:
# predicted height of the mean child - 108.31cm.
# predicted increase in height for every 10kg in weight - 27.2cm
# 95% of heights are expected to be within 2 sigma =  ~17cm of the trend line.

In [ ]:
# use to get the HPDIs
mu_pred = model_4h2_samples.posterior['mu']
height_pred = pm.sample_posterior_predictive(model_4h2_samples, model=model_4h2)

In [ ]:
x = np.linspace(howell_children.weight.min(), howell_children.weight.max(), 100)
mu = map_vals['alpha'] + map_vals['beta'] * (x - x_bar)

plt.plot(x, mu)
plt.scatter(howell_children.weight, howell_children.height, alpha=0.25)

arviz.plot_hdi(howell_children.weight, mu_pred, hdi_prob=0.89, fill_kwargs={'alpha': 0.25, 'color': '#1b9e77'})
# ppd interval for the heights
arviz.plot_hdi(howell_children.weight, height_pred.posterior_predictive['height'], hdi_prob=0.89, fill_kwargs={'alpha': 0.1, 'color': '#1b9e77'})

plt.show()

In [ ]:
# above is clearly non-linear - uncertainty intervals are as a result very wide.

In [ ]:
# 4H3 - model height vs log (weight) - what are the coefficients, and what do they mean?
with pm.Model() as model_4h3:
    # priors
    alpha = pm.Normal('alpha', mu=100, sigma=20)
    beta = pm.Normal('beta', mu=0, sigma=10)
    sigma = pm.Uniform('sigma', lower=0, upper=50)
    # model
    mu = pm.Deterministic('mu', alpha + beta * np.log(df.weight))
    # likelihood
    height=pm.Normal('height', mu=mu, sigma=sigma, observed=df.height) 
    model_4h3_samples = pm.sample(1000, tune=1000)
    map_vals_4h3 = pm.find_MAP()

                          

In [ ]:
arviz.plot_trace(model_4h3_samples, var_names=['~mu'])
plt.tight_layout()
plt.show()

In [ ]:
print('model alpha:', round(float(map_vals_4h3['alpha']), 2))
print('model beta:', round(float(map_vals_4h3['beta']), 2))
print('model sigma:', round(float(map_vals_4h3['sigma']), 2))

In [ ]:
# a ~2.7x change in weight results in 46 additional height.

In [ ]:
# use to get the HPDIs
mu_pred = model_4h3_samples.posterior['mu']
height_pred = pm.sample_posterior_predictive(model_4h3_samples, model=model_4h3)

x = np.linspace(df.weight.min(), df.weight.max(), 100)
mu = map_vals_4h3['alpha'] + map_vals_4h3['beta'] * np.log(x)
plt.plot(x, mu, color='#1b9e77')
plt.scatter(df.weight, df.height, alpha=0.25, s=2.5)

arviz.plot_hdi(df.weight, mu_pred, hdi_prob=0.97, fill_kwargs={'alpha': 0.25, 'color': '#1b9e77'})
# # ppd interval for the heights
arviz.plot_hdi(df.weight, height_pred.posterior_predictive['height'], hdi_prob=0.97, fill_kwargs={'alpha': 0.1, 'color': '#1b9e77'})

plt.show()

**4H4: plot the prior predictive distribution for the parabolic model in the chapter.**

model was:

\begin{equation}
\begin{aligned}
h_i &\sim \mathcal{N}(\mu_i, \sigma) \\
\mu_i &= \alpha + \beta_1 x_i + \beta_2 x_i^2  \\
\alpha &\sim \text{Normal}(178, 20) \\
\beta_1  &\sim \text{Log-Normal}(0,1) \\
\beta_2 &\sim \text{Normal}(0,1) \\
\sigma &\sim \text{Uniform}(0, 50)
\end{aligned}
\end{equation}


In [ ]:
num_lines = 500
alpha = alpha = np.random.normal(loc=178, scale=20, size=num_lines)
beta_1 = np.random.lognormal(mean=0, sigma=1, size=num_lines)
beta_2 = np.random.normal(loc=0, scale=1, size=num_lines)

fig, ax= plt.subplots()

num_points = 100
x = np.linspace(d2.weight.min(), d2.weight.max(), num_points)

for a, b1, b2 in zip(alpha, beta_1, beta_2):   
    ax.plot(x, [a + b1*i + b2*i**2 for i in x], alpha=0.1)

ax.set_ylabel('height')
ax.set_xlabel('weight')
ax.set_ylim([0, 300])
plt.show()

In [ ]:
# above priors are busted - below are maybe more reasonable, keeping them loose.

num_lines = 100
alpha = alpha = np.random.normal(loc=-150, scale=10, size=num_lines)
beta_1 = np.random.normal(loc=10, scale=1, size=num_lines)
beta_2 = -np.random.lognormal(mean=-2.5, sigma=0.1, size=num_lines)

fig, ax= plt.subplots()

num_points = 100
x = np.linspace(d2.weight.min(), d2.weight.max(), num_points)

for a, b1, b2 in zip(alpha, beta_1, beta_2):   
    ax.plot(x, [a + b1*i + b2*i**2 for i in x], alpha=0.1)

ax.set_ylabel('height')
ax.set_xlabel('weight')
plt.show()